# Day 1: Map with Points

> Map with point data (e.g., individual locations, points of interest, clusters). Focus on effective symbolization and density visualization.

In [42]:
import sedona.db
import os
from lonboard import Map, ScatterplotLayer

In [43]:
os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [44]:
df = sd.read_parquet(
    "/Users/matthewpowers/data/overture/base_water.parquet"
    # "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=base/type=water/"
)

In [45]:
df.to_view("base_water", True)

In [46]:
nyc = "POLYGON((-74.26 40.50, -74.26 40.92, -73.70 40.92, -73.70 40.50, -74.26 40.50))"
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{nyc}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [47]:
res = sd.sql(query)

In [48]:
res.count()

48958

In [49]:
layer = ScatterplotLayer(
    res,
    radius_min_pixels=1
)
m = Map(layer)

In [50]:
m

Map(custom_attribution='', layers=(ScatterplotLayer(radius_min_pixels=1.0, table=arro3.core.Table
+-----------…

![map](../images/day01_mp_points.png)